# GRU实现


## 1. 直接使用GRU，根据5种类型的数据，预测气温


读取数据，划分训练集和测试集


In [1]:
import pandas as pd
import torch
from Network import *

# 选取需要的列
COLUMN_INDEX = [
    "Temp (°C)",
    "Dew Point Temp (°C)",
    "Rel Hum (%)",
    "Wind Spd (km/h)",
    "Stn Press (kPa)",
]
data = pd.read_csv("Data/DataSet.csv")[COLUMN_INDEX].to_numpy()
data_tensor = torch.tensor(data, dtype=torch.float32)


# 设置随机种子以确保结果可重复
torch.manual_seed(42)
train_loader, test_loader = Split2Loaders(
    data_tensor, data_tensor, BATCHSIZE=32, RATIO=0.7, SHUFFLE=True
)


定义模型


In [2]:
class MyBiGRU(nn.Module):
    def __init__(self, INPUT_SIZE, HIDDEN_SIZE, OUTOUT_SIZE=1, NUM_LAYERS=2, SEQ=4,PARAM_INIT=True):
        super(MyBiGRU, self).__init__()        
        self.gru = nn.GRU(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS,bidirectional=True) # GRU单元
        self.fc = nn.Linear(HIDDEN_SIZE * 2, OUTOUT_SIZE)  # 因为是双向，所以乘以2
        
        # if PARAM_INIT:
        #     for name, param in self.gru.named_parameters():
        #         if name.startswith("weight"):
        #             nn.init.xavier_normal_(param)
        #         else:
        #             nn.init.zeros_(param)
        
    def forward(self, x):
        # x: (batch_size, sequence_length, input_size)
        # Initialize hidden state
        h0 = torch.zeros(2 * self.gru.num_layers, x.size(0), self.gru.hidden_size).to(x.device)
        # Forward propagate GRU
        out, _ = self.gru(x, h0)
        # Take the output from the last time step
        out = out[:, -1, :]
        # Fully connected layer
        out = self.fc(out)
        return out


训练模型


In [3]:
temp=MyBiGRU(INPUT_SIZE=5,HIDDEN_SIZE=3,OUTOUT_SIZE=2,SEQ=7)

from torchinfo import summary
summary(temp,(1,5))

# optimizer = torch.optim.Adam(MyBiGRU.parameters(), lr=0.01, weight_decay=1E-5)
# TRAIN_WITH_PROGRESS_BAR(
#     MODEL=temp,
#     NUM_EPOCHS=3,
#     OPTIMIZER=optimizer,
#     TRAIN_LOADER=train_loader,
#     TEST_LOADER=test_loader,
#     LOSS_TYPE=nn.MSELoss(),
#     DEVICE=0,
# )

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []